# Automated ML

Import all the dependencies that we will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.run import Run
import pandas as pd
from azureml.data.datapath import DataPath
from azureml.train.automl import AutoMLConfig
import joblib 
import os

## Dataset

### Overview

The dataset that we will be using for this project is the [Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) dataset from Kaggle. 

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

People with cardiovascular disease or who are at high cardiovascular risk need early detection and management wherein a machine learning model can be of great help.

**12 clinical features:**

* age - Age

* anaemia - Decrease of red blood cells or hemoglobin (boolean)

* creatinine_phosphokinase - Level of the CPK enzyme in the blood (mcg/L)

* diabetes - If the patient has diabetes (boolean)

* ejection_fraction - Percentage of blood leaving the heart at each contraction (percentage)

* high_blood_pressure - If the patient has hypertension (boolean)
  
* platelets - Platelets in the blood (kiloplatelets/mL)

* serum_creatinine - Level of serum creatinine in the blood (mg/dL)

* serum_sodium - Level of serum sodium in the blood (mEq/L)
  
* sex - Woman or man (binary)
  
* smoking - If the patient smokes or not (boolean)

* time - Follow-up period (days)

In this project we will use Azure Automated ML to make prediction on the death event based on the above mentioned clinical features.


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'new-experiment'

experiment = Experiment(ws, experiment_name)

In [3]:
print('Workspace name: '+ ws.name,
     'Azure region: '+ ws.location,
      'Subscription id: '+ ws.subscription_id,
     'Resource group: '+ ws.resource_group, sep="\n")

run = experiment.start_logging()

Workspace name: quick-starts-ws-138947
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-138947


In [4]:
ds = Dataset.get_by_name(ws, 'heart-failure-dataset')

In [5]:
df = ds.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [6]:
train_data, test_data = ds.random_split(0.9)

## Create Compute Cluster

In [7]:
cpu_cluster_name = "compute-cluster"
#Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print("Found existing cluster. Use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes =4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)
    

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

Instantiate an AutoMLConfig object for AutoML Configuration. 

The parameters used here are:

* `n_cross_validation = 3` - Since our dataset is small. We apply cross validation with 3 folds instead of train/validation data split.


* `primary_metric = 'accuracy'` - The primary metric parameter determines the metric to be used during model training for optimization. Accuracy primary metric is chosen for binary classification dataset.


* `experiment_timeout_minutes = 30` - This defines how long, in minutes, our experiment should continue to run. Here this timeout is set to 30 minutes.


* `max_concurrent_iterations = 4` - To help manage child runs and when they can be performed, we match the number of maximum concurrent iterations of our experiment to the number of nodes in the cluster. So, we get a dedicated cluster per experiment.


* `task = 'classification'` - This specifies the experiment type as classification.


*  `compute_target = cpu_cluster` -  Azure Machine Learning Managed Compute is a managed service that enables the ability to train machine learning models on clusters of Azure virtual machines. Here compute target is set to cpu_cluster which is already defined with 'STANDARD_D2_V2' and maximum nodes equal to 4.


* `training_data = train_data` - This specifies the training data to be used in this experiment which is set to train_data which is a part of the dataset uploaded to the datastore.


* `label_column_name = 'DEATH_EVENT'` - The target column here is set to DEATH_EVENT which has values 1 if the patient deceased or 0 if the patient survived.


* `featurization= 'auto'` - This indicates that as part of preprocessing, data guardrails and featurization steps are performed automatically.


* `enable_onnx_compatible_models = True` - This makes the model compatible to be converted to an ONNX model.


In [8]:
# Automl settings
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4
}

# automl config here
automl_config = AutoMLConfig(task = 'classification',
                            compute_target = cpu_cluster,
                             training_data = train_data,
                             label_column_name = 'DEATH_EVENT',
                             featurization= 'auto',
                             enable_onnx_compatible_models = True,
                             **automl_settings
                            )

In [9]:
# Submit the experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

The `RunDetails` widget shows the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-15T13:47:39.151011Z',
 'endTimeUtc': '2021-02-15T14:30:41.83033Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"new-experiment","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-138947","workspace_name":"quick-starts-ws-138947","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_clas

## Best Model

The best model from the automl experiments and all the properties of the model.



In [11]:
best_automl_run, best_automl_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [12]:
print(best_automl_run)

Run(Experiment: new-experiment,
Id: AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2_90,
Type: azureml.scriptrun,
Status: Completed)


In [13]:
print(best_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.21052631578947367,
                                                                                                silent=True,
                                                                                                subsample=1,
                                              

In [14]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
new-experiment,AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2_90,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
get_best_automl_metrics = best_automl_run.get_metrics()

for metric_name in get_best_automl_metrics:
    metric = get_best_automl_metrics[metric_name]
    print(metric_name, metric)

recall_score_micro 0.8927553927553928
average_precision_score_weighted 0.9300690057702575
AUC_micro 0.9360019291521123
precision_score_micro 0.8927553927553928
accuracy 0.8927553927553928
average_precision_score_macro 0.9121369730884568
f1_score_weighted 0.8903045630323069
average_precision_score_micro 0.9372351840732557
recall_score_weighted 0.8927553927553928
f1_score_micro 0.8927553927553928
precision_score_macro 0.8962903073687386
balanced_accuracy 0.8597336335116719
f1_score_macro 0.8738172355437784
log_loss 0.34215954979539404
AUC_macro 0.9292739642681552
precision_score_weighted 0.8942100338941298
weighted_accuracy 0.9185689335000236
matthews_correlation 0.7548584279819791
AUC_weighted 0.9292739642681552
recall_score_macro 0.8597336335116719
norm_macro_recall 0.7194672670233438
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2_90/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_bb97d84e-daaa-44dd-9bed-51494

In [41]:
# Save the best model
model = best_automl_run.register_model(model_name = 'best_automl_model', model_path = 'outputs/model.pkl', 
                                       tags = {'Training context':'Auto ML'},
                                       properties={'Accuracy': get_best_automl_metrics['accuracy']})
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-138947', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-138947'), name=best_automl_model, id=best_automl_model:4, version=4, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.8927553927553928'})


In [42]:
# List best models of HyperDrive Run and AutoML Run to compare the accuracy of the model
from azureml.core.model import Model

for model in Model.list(ws):
    print(model.name)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t',tag_name,':',tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t',prop_name,':',prop)
    print("\n")

best_automl_model
	 Training context : Auto ML
	 Accuracy : 0.8927553927553928


best_hyperdrive_model
	 Training context : Hyper Drive
	 Accuracy : 0.8




## Retrieve the Best Model's explanation

Retrieve the explanation from the `best_automl_run` which includes explanations for engineered features and raw features. 

In [18]:
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2_ModelExplain


{'runId': 'AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2_ModelExplain',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-15T14:45:06.739436Z',
 'endTimeUtc': '2021-02-15T14:54:41.71406Z',
 'properties': {'azureml.runsource': 'automl',
  'parentRunId': 'AutoML_bb97d84e-daaa-44dd-9bed-514947e543a2_90',
  '_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'b94ded6a-8126-46b9-9787-abf760229fdd',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'dependencies_versions': '{"azureml-train-automl-runtime": "1.21.0", "azureml-train-automl-client": "1.21.0", "azureml-telemetry": "1.21.0", "azureml-pipeline-core": "1.21.0", "azureml-model-management-sdk": "1.0.1b6.post1", "azureml-interpret": "1.21.0", "azureml-defaults": "1.21.0", "azureml-dataset-runtime": "1.21.0", "azureml-dataprep": "2.8.3", "azureml-dataprep-rslex": "1.6.0", "azureml-dataprep-native": "28.0.0", "azureml-core": "1.2

**Download engineered feature importance from artifact store**

Here we use `ExplanationClient` to download the engineered feature explanations from the artifact store of the `best_automl_run`.

In [19]:
from azureml.interpret import ExplanationClient
client = ExplanationClient.from_run(best_automl_run)
engineered_explanations = client.download_model_explanation(raw=False)
exp_data = engineered_explanations.get_feature_importance_dict()
exp_data

{'time_MeanImputer': 1.1933925974796407,
 'ejection_fraction_MeanImputer': 0.49060829778036813,
 'serum_creatinine_MeanImputer': 0.430819792276292,
 'age_MeanImputer': 0.24755796507703523,
 'platelets_MeanImputer': 0.18611497758261752,
 'serum_sodium_MeanImputer': 0.17610929536788705,
 'creatinine_phosphokinase_MeanImputer': 0.16102162795416042,
 'sex_ModeCatImputer_LabelEncoder': 0.10875212619747049,
 'anaemia_ModeCatImputer_LabelEncoder': 0.02589343563474471,
 'smoking_ModeCatImputer_LabelEncoder': 0.020113523077104706,
 'diabetes_ModeCatImputer_LabelEncoder': 0.01891876922046284,
 'high_blood_pressure_ModeCatImputer_LabelEncoder': 0.004459226613227432}

**Download raw feature importance from artifact store**

`ExplanationClient` is used to download the raw feature explanations from the artifact store of the `best_automl_run`.

In [20]:
client = ExplanationClient.from_run(best_automl_run)
engineered_explanations = client.download_model_explanation(raw=True)
exp_data = engineered_explanations.get_feature_importance_dict()
exp_data

{'time': 1.1933925974796407,
 'ejection_fraction': 0.49060829778036813,
 'serum_creatinine': 0.430819792276292,
 'age': 0.24755796507703523,
 'platelets': 0.18611497758261752,
 'serum_sodium': 0.17610929536788705,
 'creatinine_phosphokinase': 0.16102162795416042,
 'sex': 0.10875212619747049,
 'anaemia': 0.02589343563474471,
 'smoking': 0.020113523077104706,
 'diabetes': 0.01891876922046284,
 'high_blood_pressure': 0.004459226613227432}

## Retrieve the Best ONNX Model

Then we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing. Overloads on `get_output` allows to retrieve the best run and fitted model for any logged metric or for a particular iteration.

Set the parameter `return_onnx_model = True` to retrieve the best ONNX model, instead of the Python model.

In [21]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)

## Save the best ONNX model

In [22]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

## Predict with the ONNX model

In [24]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    test_df = test_data.to_pandas_dataframe()
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(best_run)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(test_df)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

[1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[0.04875111 0.9512489 ]
 [0.1632092  0.83679086]
 [0.22066556 0.7793344 ]
 [0.2586167  0.7413833 ]
 [0.10550058 0.8944995 ]
 [0.12873255 0.87126744]
 [0.16924183 0.8307582 ]
 [0.13441235 0.86558765]
 [0.28711382 0.7128861 ]
 [0.14024073 0.85975933]
 [0.4622667  0.5377333 ]
 [0.52251655 0.47748342]
 [0.93948853 0.0605114 ]
 [0.8739956  0.12600438]
 [0.95519173 0.04480835]
 [0.9438979  0.05610217]
 [0.9401381  0.05986191]
 [0.92458653 0.0754135 ]
 [0.9452403  0.05475964]
 [0.97015107 0.02984898]
 [0.868021   0.13197903]
 [0.9560322  0.0439679 ]
 [0.9266303  0.07336968]
 [0.97783405 0.02216594]
 [0.96971595 0.03028409]
 [0.5544214  0.44557852]
 [0.9794257  0.02057419]
 [0.91772723 0.08227274]]


In [44]:
cpu_cluster.delete()